# Processing I: from XDF to raw files

As described in the preregistration of methods (@REF), the data has been collected using LabStreamLayer, allowing for recording of precisely synchronized streams. These streams include:
- MicStream (16kHz)
- WebcamFrameStream (ca. 60 Hz)
- MarkerStream (sent from custom buttonbox)
- BalanceBoardStream (500 Hz)

LSL outputs single .xdf file that contains all the data streams. This script serves for extracting the streams and creating raw files for further processing.

This script therefore creates each stream as a csv file (one for whole session, and one for ech trial), and  additionaly creates audio and video files for each trial.

Additionally, in this script, we:
- correct faulty delimitations of trials
- align 16kHz audio and additional 48kHz audio that has been recorded outside of LSL, and cut this aligned audio to trial sized files
- combine video and audio


In [ ]:
# import packages

import os
import pyxdf
import glob
import pandas as pd
import numpy as np
import wave, struct, math, random
from scipy.io import wavfile
import noisereduce as nr
import pandas as pd
import cv2
import random
import json

# set folders
curfolder = os.getcwd()
datafolder = curfolder+'\\data\\'
errorlogs = datafolder+'\\error_logs\\'
experiment_to_process = curfolder + '\\..\\00_RAWDATA\\'
outputfolder = datafolder+'\\Data_processed\\CsvDataTS_raw\\' # outputfolder raw
trialfolder = datafolder+'\\Data_processed\\Data_trials\\' # outputfolder trialbased

# map all the folders in the target folder
datafolders = glob.glob(experiment_to_process+'*\\')

# extract the folder IDs
datafolders_id = [x.split('\\')[-2] for x in datafolders]

# print(curfolder)
# print(targetfolder)
print(datafolders_id)

# identify all xdf files and all the associated triallist info
xdffiles = []
trialdatas = []

for i in datafolders_id:
    file = glob.glob(experiment_to_process+i+'\\*.xdf')
    trialfile = experiment_to_process+i+'\\'+i+'_results'+'.csv'
    trialdatas.append(trialfile)
    xdffiles.extend(file)

# these are the xdf files we need to process
print(xdffiles)
print(trialdatas)

c:\Users\kadava\AppData\Local\anaconda3\envs\TSPROCESS\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


['0_1', '0_2']
['e:\\FLESH_ContinuousBodilyEffort\\01_XDF_processing\\..\\00_RAWDATA\\0_1\\session_S001.xdf', 'e:\\FLESH_ContinuousBodilyEffort\\01_XDF_processing\\..\\00_RAWDATA\\0_2\\session_S002.xdf']
['e:\\FLESH_ContinuousBodilyEffort\\01_XDF_processing\\..\\00_RAWDATA\\0_1\\0_1_results.csv', 'e:\\FLESH_ContinuousBodilyEffort\\01_XDF_processing\\..\\00_RAWDATA\\0_2\\0_2_results.csv']


# Extracting streams from XDF file

Using `pyxdf` package, we can extract the streams from the XDF file.

First, we extract streams for the whole session. Then, we cut these streams into trial-sized chunks, based on the marker stream that indicates the start of each trial. Moreover, we connect the trial with information from the PsychoPy log file, which includes metadata about the trial (i.e., concept, correction number, etc.)

(Note that this code takes some time to execute.)


In [7]:
# audio write function
def to_audio(fileloc, timeseries, samplerate = 16000, channels = 1):
    if timeseriestype == 'Mic': 
        obj = wave.open(fileloc,'w')
        obj.setnchannels(channels) # mono
        obj.setsampwidth(2)
        obj.setframerate(float(samplerate))
        for i in timeseries:
            data = struct.pack('<h', int(i[0]))
            obj.writeframesraw( data )
        obj.close()

# function to retrieve closest value
def find_closest_value_and_retrieve(df, target_number, target_column, retrieve_column):
    # get the absolute differences between a value in column and the target number
    differences = abs(df[target_column] - target_number)
    
    # find the index of the minimum difference
    min_difference_index = differences.idxmin()
    
    # retrieve the closest value in the target column
    closest_value = df.loc[min_difference_index, target_column]
    #print(closest_value)
    # retrieve the corresponding value from the column
    result_value = df.loc[min_difference_index, retrieve_column]

    return result_value

In [ ]:
errorlist = []

for dat in datafolders_id:
    print('Loading in data from participant: ' + dat)
    trialdata = pd.read_csv(experiment_to_process+dat+'\\'+dat+'_results.csv', sep=",")
    #print(trialdata)
    
    # get the xdf file
    files = glob.glob(experiment_to_process+dat+'\\*.xdf')
    streams, header = pyxdf.load_xdf(files[0])   
    # we go through each stream and save it as a csv, 
    # if it's Mic stream, also as a .wav file
    for stream in streams:
        timeseriestype = stream['info']['name'][0]
        print(timeseriestype)
        samplerate = round(float(stream['info']['nominal_srate'][0]))
        # in the xdf loop over the streams and save it as csv if not yet exists
        channelcount = stream['info']['channel_count'][0]
        print('working on stream: ' + timeseriestype + '  with a channel count of ' + str(channelcount) +'\n and a sampling rate of ' + str(samplerate))
        timevec = stream['time_stamps']
        timeseries = stream['time_series']
        matrix_aux = np.vstack([np.transpose(timevec),np.transpose(timeseries)])
        matrix = np.transpose(matrix_aux)
        df_lab = pd.DataFrame(matrix)
        df_lab.to_csv(outputfolder+dat+'_'+timeseriestype+'_nominal_srate'+str(samplerate)+'.csv',index=False)
        #for audio data also create a wav file
        if timeseriestype == 'Mic':
            wavloc = outputfolder+'Audio/'+dat+'_'+timeseriestype+'_nominal_srate'+str(samplerate)+'.wav'
            to_audio(wavloc, timeseries)
            # load data
            rate, data = wavfile.read(wavloc)
            # perform noise reduction
            reduced_noise = nr.reduce_noise(y=data, sr=rate, n_std_thresh_stationary=1.5,stationary=True)
            wavloc2 = outputfolder+'Audio/'+dat+'_'+timeseriestype+'_nominal_srate'+str(samplerate)+'_denoised.wav'
            wavfile.write(wavloc2, rate, reduced_noise)

        print('done with processing a complete time series and audio data')
        print('we will now start making trial snipped data') 

        # load the MyMarkerStream
        markers = pd.read_csv(outputfolder+dat+'_MyMarkerStream_nominal_srate0.csv')
        
        # cut all timeseries to trial level based on the markers
        if timeseriestype != 'MyMarkerStream':
            
            beginlist = []
            endlist = []
            timestamps = []
            timestamps_2 = []
            tpose_starts = []
            tpose_ends = []

            # iterate over markers and save times of trial starts and ends
            for row in markers.iterrows():
                # Accessing positional elements explicitly with .iloc
                if 'Trial_start' in row[1].iloc[1] or 'Practice trial starts' in row[1].iloc[1]:
                    beginlist.append(row[1].iloc[0])
                if 'Trial_end' in row[1].iloc[1] or 'Practice trial ends' in row[1].iloc[1]:
                    endlist.append(row[1].iloc[0])
                if 'Experiment_start' in row[1].iloc[1]:
                    timestamps.append(row[1].iloc[0])
                if 'New block starts' in row[1].iloc[1]:
                    timestamps_2.append(row[1].iloc[0])
                if 'Tpose starts' in row[1].iloc[1]:
                    tpose_starts.append(row[1].iloc[0])
                if 'Tpose ends' in row[1].iloc[1]:
                    tpose_ends.append(row[1].iloc[0])
                    
            # converting coefficient for lsl to psychopy time
            exp_start_pp = float(trialdata['exp_start'][0])
            block_start_pp = float(trialdata['block_start'][0])
            # get to lsl_to_pp coefficient
            if timestamps != []:
                lsl_to_pp = timestamps[0] - exp_start_pp
            else:
                lsl_to_pp = timestamps_2[0] - block_start_pp
        
            # now we can proceed to cutting   
            for i in range(len(beginlist)):
                # prepare the range of the trial
                begin = beginlist[i]
                end = endlist[i]
                indices = (df_lab.loc[:,0] > begin) & (df_lab.loc[:,0] < end)
                beginst = min(df_lab.loc[:,0]) #start time of the timeseries
                endst = max(df_lab.loc[:,0])  #end time of the timeseries
                subset = df_lab.loc[indices, :]
                # convert the beginst to psychopy time
                beginst_pp = begin - lsl_to_pp
                # now find in trialdata the closest time to the beginst_pp to gather info
                # whether it is practice or trial
                practice = find_closest_value_and_retrieve(trialdata, beginst_pp, 'trial_start', 'practice')
                if practice == 'practice':
                    trialtype = 'pr'
                else:
                    trialtype = 'trial'
                
                # which participant it is
                cycle = find_closest_value_and_retrieve(trialdata, beginst_pp, 'trial_start', 'cycle')
                if cycle == 0:
                    participant = 'p0'
                else:
                    participant = 'p1'

                # what concept it is
                word = find_closest_value_and_retrieve(trialdata, beginst_pp, 'trial_start', 'word')
                # modality
                modality = find_closest_value_and_retrieve(trialdata, beginst_pp, 'trial_start', 'modality')
                # correction, if applicable
                correction_info = find_closest_value_and_retrieve(trialdata, beginst_pp, 'trial_start', 'correction')
                if correction_info == 0:
                    correction = '_c0'
                elif correction_info == 1:
                    correction = '_c1'
                elif correction_info == 2:
                    correction = '_c2'
                else:
                    correction = ''
                
                # continue saving
                if(len(subset.axes[0])<2):
                    errorlist.append(dat + " for "+ timeseriestype + " for trial " + str(i) + 'NO DATA WITHIN RANGE...')
                if(len(subset.axes[0])>2):
                     # save subset to csv
                      subset.to_csv(trialfolder+dat+'_'+trialtype+'_'+ str(i) +'_'+timeseriestype+'_nominal_srate'+str(samplerate)+'_'+participant+'_'+word+'_'+modality+correction+'.csv', index=False)
                      if timeseriestype == 'Mic':
                            wavloc = trialfolder+'Audio/'+dat+'_'+trialtype+'_'+ str(i) +'_'+timeseriestype+'_nominal_srate'+str(samplerate)+'_'+participant+'_'+word+'_'+modality+correction+'.wav'
                            to_audio(wavloc, timeseries[indices])
                          # also apply denoising
                            reduced_noiseclip = reduced_noise[indices]
                            wavloc2 = trialfolder+'Audio/'+dat+'_'+trialtype+'_'+ str(i) +'_'+timeseriestype+'_nominal_srate'+str(samplerate)+'_'+participant+'_'+word+'_'+modality+correction+'_denoised.wav'
                            wavfile.write(wavloc2, rate, reduced_noiseclip)
            
            # get information about the tpose for camera
            if timeseriestype == 'MyWebcamFrameStream':
                for i in range(len(tpose_starts)):
                    begin = tpose_starts[i]
                    end = tpose_ends[i]
                    indices = (df_lab.loc[:,0] > begin) & (df_lab.loc[:,0] < end)
                    beginst = min(df_lab.loc[:,0])
                    endst = max(df_lab.loc[:,0])
                    subset = df_lab.loc[indices, :]
                    # save subset to csv
                    subset.to_csv(trialfolder+dat+'_'+'tpose_'+ str(i) +'_'+timeseriestype+'_nominal_srate'+str(samplerate)+'.csv', index=False)
        
        # after every stream we'll save the error log
        errors = pd.DataFrame(errorlist, columns=['file_error'])
        # get todays date
        today = pd.Timestamp("today").strftime("%Y_%m")
        file_path = errorlogs+'error_log_cuttingtrails' + today + '.csv'
        errors.to_csv(file_path, index=False) 
        
print('Were done: proceed to snipping videos to triallevel')

Loading in data from participant: 0_1
MyMarkerStream
working on stream: MyMarkerStream  with a channel count of 1
 and a sampling rate of 0
done with processing a complete time series and audio data
we will now start making trial snipped data
MyWebcamFrameStream
working on stream: MyWebcamFrameStream  with a channel count of 1
 and a sampling rate of 500
done with processing a complete time series and audio data
we will now start making trial snipped data
BalanceBoard_stream
working on stream: BalanceBoard_stream  with a channel count of 4
 and a sampling rate of 500
done with processing a complete time series and audio data
we will now start making trial snipped data
Mic
working on stream: Mic  with a channel count of 1
 and a sampling rate of 16000
done with processing a complete time series and audio data
we will now start making trial snipped data
Loading in data from participant: 0_2
MyMarkerStream
working on stream: MyMarkerStream  with a channel count of 1
 and a sampling rate o

Here is an audio example

In [ ]:
from IPython.display import Audio

wavfiles = glob.glob(trialfolder+'Audio\\*.wav')
# keep only the denoised files
wavfiles = [x for x in wavfiles if 'denoised' in x]
# pick random one
randomfile = random.choice(wavfiles)

Audio(randomfile)

Now we have raw data for each trial in csv and wav, but we still need to write videos for each trial. We will use the trial-cut files to access the range of frames in the original raw video file and cut it out from it


In [ ]:
errorlist = []

# this is a folder with timeseries
tsfolder = datafolder+'\\Data_processed\\Data_trials\\'
# this is where the raw long video is
videofolder = datafolder+'raw_data\\'
#print(tsfolder)
#print(videofolder)

# loop through the csv's in tsfolder that has string 'MyWebcamStream' in name
for file in os.listdir(tsfolder):
    print(file)
    if 'MyWebcamFrameStream' in file:
        print('Now processing file '+ file)
        # if it is a tpose file, we skip it for now
        if 'tpose' in file:
            continue

        # the name looks like this 0_1_trial_0_MyWebcamFrameStream_nominal_srate500_p0_bitter_geluiden.csv

        dyadIndex = file.split('_')[0]   # this is dyad number
        partIndex = file.split('_')[1]   # this is part of the session
        sessionIndex = dyadIndex + '_' + partIndex # this is the session index
        trialIndex = file.split('_')[3] # this is trial number
        participant = file.split('_')[7] # this is participant 0/1
        word = file.split('_')[8] # this is the concept
        modality = file.split('_')[9].split('.')[0] # this is the modality

        # assess the correction
        if 'c0' in file:
            correction = '_c0'
        elif 'c1' in file:
            correction = '_c1'
        elif 'c2' in file:
            correction = '_c2'
        else:
            correction = ''


        # assess the trial type
        if 'pr' in file:
            trialtype = 'pr'
        else:
            trialtype = 'trial'

        trialdata = pd.read_csv(tsfolder+file)
        #print(trialdata)
        videolong = videofolder+sessionIndex+'\\'+sessionIndex+'-video.avi' # this is the long video 
        begin_time = trialdata['0'].min() # begin time of the trial
        #print(begin_time)
        end_time = trialdata['0'].max() # end time of the trial
        #print(end_time)
        # get the begin and end frame
        begin_frame = trialdata['1'].min().astype(int)
        #print(begin_frame)
        end_frame = trialdata['1'].max().astype(int)
        #print(end_frame)
        totframes = end_frame-begin_frame # total number of frames in the trial
        #print(totframes)
        frames = range(begin_frame, end_frame) # get all the frames in trial
        #print(frames)
        # load in the long video
        print('Loading the original video')
        capture = cv2.VideoCapture(videolong) 

        # what is the original fps of the video
        originalfps = round((totframes/(end_time-begin_time)),3)
        print('original fps: '+str(originalfps))
        
        # this is the location where the video will be saved
        vidloc = trialfolder+sessionIndex+'_'+trialtype+'_'+ str(trialIndex) +'_'+participant+'_'+word+'_'+modality+correction+'_video_raw'+'.avi'
        # codec for the video
        fourcc = cv2.VideoWriter_fourcc(*'XVID')
        # get the frame width
        frameWidth = capture.get(cv2.CAP_PROP_FRAME_WIDTH)
        # get the frame height
        frameHeight = capture.get(cv2.CAP_PROP_FRAME_HEIGHT)

        # start writing video
        print('Starting to write the video')
        #out = ffmpegcv.VideoWriter(vidloc, fourcc, fps = originalfps, frameSize = (int(frameWidth), int(frameHeight)))
        out = cv2.VideoWriter(vidloc, fourcc, fps = originalfps, frameSize = (int(frameWidth), int(frameHeight)))

        print('Looping over frames')

        # index the frames of the current trial
        for fra in frames:
            capture.set(cv2.CAP_PROP_POS_FRAMES, fra)
            ret, frame = capture.read()
            if ret:
                out.write(frame)
            if not ret:
                print('a frame was dropped: ' + str(fra))
            
        capture.release()
        out.release()
        
        print('Video is done')

print('All done!')


0_1_pr_1_MyWebcamFrameStream_nominal_srate500_p0_dansen_geluiden_corrected.csv
Now processing file 0_1_pr_1_MyWebcamFrameStream_nominal_srate500_p0_dansen_geluiden_corrected.csv
Loading the original video
original fps: 59.739
Starting to write the video
Looping over frames
Video is done
0_1_tpose_0_MyWebcamFrameStream_nominal_srate500_video_raw.avi
Now processing file 0_1_tpose_0_MyWebcamFrameStream_nominal_srate500_video_raw.avi
0_1_pr_10_MyWebcamFrameStream_nominal_srate500_p1_auto_geluiden_corrected.csv
Now processing file 0_1_pr_10_MyWebcamFrameStream_nominal_srate500_p1_auto_geluiden_corrected.csv
Loading the original video
original fps: 59.839
Starting to write the video
Looping over frames
Video is done
0_1_pr_18_MyWebcamFrameStream_nominal_srate500_p0_snel_combinatie_corrected.csv
Now processing file 0_1_pr_18_MyWebcamFrameStream_nominal_srate500_p0_snel_combinatie_corrected.csv
Loading the original video
original fps: 59.88
Starting to write the video
Looping over frames
Video

In [5]:
from IPython.display import Video

# pick one video in tsfolder
avifiles = glob.glob(trialfolder+'*.avi')
sample = random.choice(avifiles)

# adjust the size
Video(sample, width=450, height=250)


Now we also write the video of tpose, using the same code but looking only for csv files with tpose in the name


In [28]:
errorlist = []

# this is a folder with timeseries
tsfolder = datafolder+'Data_processed\\Data_trials\\'
# this is where the raw long video is
videofolder = experiment_to_process
#print(tsfolder)
#print(videofolder)

# loop through the csv's in tsfolder that has string 'MyWebcamStream' in name
for file in os.listdir(tsfolder):
    if 'MyWebcamFrameStream' in file and 'tpose' in file and 'csv' in file:
        print('Now processing file '+file)
        print(file)
        # get the filename 
        filename = file.split('.')[0]
        sessionIndex = filename.split('_')[0] + '_' + filename.split('_')[1]
        print(sessionIndex)
        videolong = videofolder+sessionIndex+'\\'+sessionIndex+'-video.avi' # this is the long video
        trialdata = pd.read_csv(tsfolder+file)         # this is the csv file

        begin_time = trialdata['0'].min() # begin time of the trial
        #print(begin_time)
        end_time = trialdata['0'].max() # end time of the trial
        #print(end_time)
        # get the begin and end frame
        begin_frame = trialdata['1'].min().astype(int)
        print(begin_frame)
        end_frame = trialdata['1'].max().astype(int)
        print(end_frame)
        totframes = end_frame-begin_frame # total number of frames in the trial
        #print(totframes)
        frames = range(begin_frame, end_frame) #get all the frames in trial
        #print(frames)
        
        #load in the long video
        print('Loading the original video')
        capture = cv2.VideoCapture(videolong) 

        originalfps = round((totframes/(end_time-begin_time)),3)
        print('original fps: '+str(originalfps))
        
        # this is the location where the video will be saved
        vidloc = trialfolder+filename+'_video_raw'+'.avi'
        # codec for the video
        fourcc = cv2.VideoWriter_fourcc(*'XVID')
        # get the frame width
        frameWidth = capture.get(cv2.CAP_PROP_FRAME_WIDTH)
        # get the frame height
        frameHeight = capture.get(cv2.CAP_PROP_FRAME_HEIGHT)
        # start writing video
        print('Starting to write the video')
        #out = ffmpegcv.VideoWriter(vidloc, fourcc, fps = originalfps, frameSize = (int(frameWidth), int(frameHeight)))
        out = cv2.VideoWriter(vidloc, fourcc, fps = originalfps, frameSize = (int(frameWidth), int(frameHeight)))

        print('Looping over frames')

        # index the frames of the current trial
        for fra in frames:
            capture.set(cv2.CAP_PROP_POS_FRAMES, fra)
            ret, frame = capture.read()
            if ret:
                out.write(frame)
                
            if not ret:
                print('a frame was dropped: ' + str(fra))
            
        capture.release()
        out.release()

print('All done!')


Now processing file 0_1_tpose_0_MyWebcamFrameStream_nominal_srate500.csv
0_1_tpose_0_MyWebcamFrameStream_nominal_srate500.csv
0_1
12508
12705
Loading the original video
original fps: 59.601
Starting to write the video
Looping over frames
Now processing file 0_1_tpose_1_MyWebcamFrameStream_nominal_srate500.csv
0_1_tpose_1_MyWebcamFrameStream_nominal_srate500.csv
0_1
42270
42506
Loading the original video
original fps: 60.169
Starting to write the video
Looping over frames
All done!


Here is a video example

In [7]:
from IPython.display import Video

# pick one video in tsfolder
avifiles = glob.glob(trialfolder+'*.avi')
# keep only file with tpose in name
avifiles = [x for x in avifiles if 'tpose' in x]
sample = random.choice(avifiles)

# adjust the size
Video(sample, width=450, height=250)

# Correcting some trials

Note that the trials are meant to be delimited by button box markers that are triggered by participants locking their hand in the beginning and in the end of each performance. However, due to occasional errors (e.g., participants forgot to lock their hands), we need to check all trials, and correct the ones that are not properly delimited.

For each session, we create a separate csv file where we note the correct beginning and end frame (as shown in the video itself) of the wrongly delimited trials.

In [8]:
# audio write function
def to_audio(fileloc, timeseries, samplerate = 16000, channels = 1):
    obj = wave.open(fileloc,'w')
    obj.setnchannels(channels) # mono
    obj.setsampwidth(2)
    obj.setframerate(float(samplerate))
    for i in timeseries:
        data = struct.pack('<h', int(i))
        obj.writeframesraw( data )
    obj.close()

def correct_file(i, begin, end, errorlog, ts, stream):
    # get the start and end of the corrected range of frames
    # begin = error['frame_begin'][i]
    # end = error['frame_end'][i]
    if stream == 'video':
        indexcol = 1
    else:
        indexcol = 0
    # cut from the ts everything that is below start and above end
    indices = (ts.iloc[:,indexcol] > begin) & (ts.iloc[:,indexcol] < end)
    #beginst = min(ts.iloc[:,1]) #start time of the timeseries
    #endst = max(ts.iloc[:,1])  #end time of the timeseries
    subset = ts.loc[indices, :]
    # save also the time of the first and last frame as new variables
    if stream == 'video':
        errorlog['begin_time'][i] = subset.iloc[0,1]
        errorlog['end_time'][i] = subset.iloc[len(subset)-1,1]

    #print(errorlog['end_time'][i])
    # save the new timeseries using the path of the video column with _corrected as appended
    file_name = errorlog[stream][i]
    # replace .csv by _corrected.csv
    file_name = file_name.replace('.csv', '_corrected.csv')
    return errorlog, file_name, subset





In [ ]:
datafolder = curfolder + '/data/Data_processed/Data_trials/'
errorfolder = datafolder + 'Handlock_error/' # here we store all the corrected frame ranges
timeseries = curfolder + '/data/Data_processed/CsvDataTS_raw/'

errorfiles = glob.glob(errorfolder+'*.csv')

for file in errorfiles:
    # load the df
    error = pd.read_csv(file)
    session = file.split('\\')[-1].split('_')[0]

    # make new columns audio, video and bb
    error['audio'] = 0
    error['video'] = 0
    error['bb'] = 0

    # now let's find corresponding wrong files in audio and video
    audio = datafolder + 'Audio/'
    csv = glob.glob(datafolder + '*.csv')

    # loop over the error words
    for i in range(len(error)):
        # get the word
        word = error['word'][i]
        # get the part
        part = error['part'][i]
        # if the correction is not NaN, get it and transform to integer
        if pd.isna(error['correction'][i]) == False:
            correction = int(error['correction'][i])
            # if correction is 0, it is c0
            if correction == 0:
                correction = 'c0'
            elif correction == 1:
                correction = 'c1'
            else:
                correction = 'c2'
            
            part = '2'
            sessionID = session + '_' + part

        else:
            correction = ''
            part = '1'
            sessionID = session + '_' + part

        # loop over the csv files
        for j in range(len(csv)):
            # if the word is in the csv file
            if word in csv[j] and sessionID in csv[j] and correction in csv[j]:
                # get the corresponding csv file
                csv_file = csv[j]
                # if the file has Mic in it, save its name to column audio
                if 'Mic' in csv_file:
                    error['audio'][i] = csv_file
                # if the file has Webcam in it, save its name to column video
                elif 'Webcam' in csv_file:
                    error['video'][i] = csv_file
                # if BalanceBoard in it, save its name to column bb
                elif 'BalanceBoard' in csv_file:
                    error['bb'][i] = csv_file


    # create begin_time and end_time columns
    error['begin_time'] = 0
    error['end_time'] = 0

    ########################## re-cutting csv files ##############################

    # now we will get to each timeseries, and cut out the part that is wrong
    csv_ts = glob.glob(timeseries + '*.csv')
    # csv_webcams only
    webcam_ts = [x for x in csv_ts if 'Webcam' in x]
    mic_ts = [x for x in csv_ts if 'Mic_nominal' in x]
    bb_ts = [x for x in csv_ts if 'BalanceBoard' in x]
    
    # loop over the error words
    for i in range(len(error)):
        print(error['word'][i])
        # if the session is 0_1 load 0_1_MyWebcamFrameStream
        if error['part'][i] == 1:
            sessionID = session + '_1'
            webcam_ts = timeseries + sessionID + '_MyWebcamFrameStream_nominal_srate500.csv'
            ts_w = pd.read_csv(webcam_ts)
            mic_ts = timeseries + sessionID + '_Mic_nominal_srate16000.csv'
            ts_m = pd.read_csv(mic_ts)
            bb_ts = timeseries + sessionID + '_BalanceBoard_stream_nominal_srate500.csv'
            ts_bb = pd.read_csv(bb_ts)
        elif error['part'][i] == 2:
            sessionID = session + '_2'
            # webcam_ts = timeseries + session + '_2_MyWebcamFrameStream_nominal_srate500.csv'
            # ts = pd.read_csv(webcam_ts)
            mic_ts = timeseries + sessionID + '_Mic_nominal_srate16000.csv'
            bb_ts = timeseries + sessionID + '_BalanceBoard_stream_nominal_srate500.csv'
            # get the start and end of the corrected range of frames
            begin = error['frame_begin'][i]
            end = error['frame_end'][i]

            ############ this is a bug of dyad 0 ####################
            if begin < 91500: 
                webcam_ts = timeseries + '0_1_MyWebcamFrameStream_nominal_srate500.csv'
                ts_w = pd.read_csv(webcam_ts)
            else:
                webcam_ts = timeseries + sessionID + '_MyWebcamFrameStream_nominal_srate500.csv'
                ts_w = pd.read_csv(webcam_ts)
            ##########################################################

        # Video correction 
        begin = error['frame_begin'][i]
        end = error['frame_end'][i]
        error, file_name, subset = correct_file(i, begin, end, error, ts_w, 'video')
        if len(subset) == 0:
            print('This file is empty: ' + error['video'][i])
        else:
            # save also the begin_time and end_time
            error['begin_time'][i] = subset.iloc[0,0]
            error['end_time'][i] = subset.iloc[len(subset)-1,0]
            subset.to_csv(file_name, index = False)  

        # Audio correction
        begin_time = error['begin_time'][i]
        end_time = error['end_time'][i]
        error, filename, subset = correct_file(i, begin_time, end_time, error, ts_m, 'audio')
        if len(subset) == 0:
            print('This file is empty: ' + error['audio'][i])
        else:
            subset.to_csv(filename, index = False)
            
        # BalanceBoard correction
        error, filename, subset = correct_file(i, begin_time, end_time, error, ts_bb, 'bb')
        if len(subset) == 0:
            print('This file is empty: ' + error['bb'][i])
        else:
            subset.to_csv(filename, index = False)

    ################### rewriting audio and video ############################

    # get all the csv files with _corrected in it
    correct = glob.glob(datafolder + '*_corrected.csv')
    audio = datafolder + 'Audio/'
        
    # loop over them 
    for i in range(len(correct)):
        print(correct[i])
        sessionID = correct[i].split('\\')[-1].split('_')[0] + '_' + correct[i].split('\\')[-1].split('_')[1]

    # if it has Mic in it, create an audio
        if 'Mic' in correct[i]:
            file_name = correct[i]
            # take only the last part of the path
            file_name = file_name.split('\\')[-1]
            # replace _corrected.csv by .wav
            file_name = file_name.replace('.csv', '.wav')          
            print(file_name)
            wavloc = audio+file_name
            ts = pd.read_csv(correct[i])
            # make second column into a list
            sound = ts.iloc[:,1].tolist()
            to_audio(wavloc, sound)
            # load data    
            rate, data = wavfile.read(wavloc)
            # perform noise reduction
            reduced_noise = nr.reduce_noise(y=data, sr=rate, n_std_thresh_stationary=1.5,stationary=True)
            file_name2 = file_name.replace('.wav', '_denoised.wav')
            wavloc2 = audio + file_name2
            wavfile.write(wavloc2, rate, reduced_noise)

        elif 'Webcam' in correct[i]:
            print('This is a video')
            ts = pd.read_csv(correct[i])
            #if '0_1' in correct[i]:
            videolong = experiment_to_process+ '\\' + sessionID + '\\' + sessionID + '-video.avi'
            # else:
            #     videolong = experiment_to_process + '\\0_2\\0_2-video.avi'
            
            print(videolong)
            begin_time = ts['0'].min() # begin time of the trial
            print(begin_time)
            end_time = ts['0'].max() # end time of the trial
            print(end_time)
            # get the begin and end frame
            begin_frame = ts['1'].min().astype(int)
            print(begin_frame)
            end_frame = ts['1'].max().astype(int)
            print(end_frame)
            totframes = end_frame-begin_frame # total number of frames in the trial
            print(totframes)
            frames = range(begin_frame, end_frame) #get all the frames in trial
            print(frames)
            #load in the long video
            print('Loading the original video')
            capture = cv2.VideoCapture(videolong) 
            originalfps = round((totframes/(end_time-begin_time)),3)
            print('original fps: '+str(originalfps))
            
            # filename
            filename = correct[i].split('\\')[-1]
            # replace MyWebcamFrameStream_nominal_srate500_ with ''
            filename = filename.replace('MyWebcamFrameStream_nominal_srate500_', '')
            # replace csv. with ''
            filename = filename.replace('.csv', '')
            # this is the location where the video will be saved
            vidloc = datafolder + filename + '_video_raw' + '.avi'
            # codec for the video
            fourcc = cv2.VideoWriter_fourcc(*'XVID')
            # get the frame width
            frameWidth = capture.get(cv2.CAP_PROP_FRAME_WIDTH)
            # get the frame height
            frameHeight = capture.get(cv2.CAP_PROP_FRAME_HEIGHT)
            # start writing video
            print('Starting to write the video')
            #out = ffmpegcv.VideoWriter(vidloc, fourcc, fps = originalfps, frameSize = (int(frameWidth), int(frameHeight)))
            out = cv2.VideoWriter(vidloc, fourcc, fps = originalfps, frameSize = (int(frameWidth), int(frameHeight)))
            #frame_count = 0
            #print(len(capture))
            print('Looping over frames')

            # index the frames of the current trial
            for fra in frames:
                capture.set(cv2.CAP_PROP_POS_FRAMES, fra)
                ret, frame = capture.read()
                if ret:
                    # display th loading image with im.show() to see if the frames are loaded correctly
                    #cv2.imshow('frame',frame)
                    #cv2.waitKey(1000)
                    out.write(frame)
                    
                if not ret:
                    print('a frame was dropped: ' + str(fra))
                
            capture.release()
            out.release()
        else:
            print('BB do not need any processing')



C:\Users\kadava\AppData\Local\Temp\ipykernel_18780\367154011.py:57: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  error['video'][i] = csv_file
C:\Users\kadava\AppData\Local\Temp\ipykernel_18780\367154011.py:57: SettingWithCopyWarning: 
A val

dansen


C:\Users\kadava\AppData\Local\Temp\ipykernel_18780\3709239627.py:27: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  errorlog['begin_time'][i] = subset.iloc[0,1]
C:\Users\kadava\AppData\Local\Temp\ipykernel_18780\3709239627.py:27: SettingWithC

langzaam
auto
snel
bitter
walgen
vliegtuig
Empty DataFrame
Columns: [0, 1]
Index: []
vrouw
Empty DataFrame
Columns: [0, 1]
Index: []
vallen


KeyboardInterrupt: 

# Concatenating audio and video, using ffmpeg

We do not necessarilly need to have video and audio combined, but we do want to check whether they are synchronized (they should be, but it's always good to do a sanity check)


In [11]:
import ffmpeg

wavloc = trialfolder+'Audio_48\\'
audiovideo = datafolder+'Data_processed\\AudioVideo\\'

if not os.path.exists(wavloc):
    print(f"Directory not found: {wavloc}")

# loop over Audio files
for file in os.listdir(wavloc):
    print(file)
    if '0_1' in file and 'wav' in file and 'corrected' not in file:
        print('Now processing file '+file)
        dyadIndex = file.split('_')[0]   # this is dyad number
        partIndex = file.split('_')[1]   # this is part of the session
        sessionIndex = dyadIndex + '_' + partIndex # this is the session index
        trialIndex = file.split('_')[3] # this is trial number

        participant = file.split('_')[7] # this is participant 0/1
        word = file.split('_')[8] # this is the word
        modality = file.split('_')[9] # this is the modality
        # put the .csv away from the modality
        modality = modality.split('.')[0]

        if 'c0' in file:
            correction = '_c0'
        elif 'c1' in file:
            correction = '_c1'
        elif 'c2' in file:
            correction = '_c2'
        else:
            correction = ''
        
        if 'pr' in file:
            trialtype = 'pr'
        else:
            trialtype = 'trial'
        #load in the audio
        print('Loading the audio')
        audio_path = os.path.join(wavloc, file)
        if not os.path.exists(audio_path):
            print(f"Audio file not found: {audio_path}")
        # input the video with ffmpg
        input_audio = ffmpeg.input(audio_path)
        print(input_audio)
        #load in the video with matchich trialIndex and SessionIndex
        print('Loading the video')
        video_path = os.path.join(trialfolder, f"{sessionIndex}_{trialtype}_{trialIndex}_{participant}_{word}_{modality}{correction}_video_raw.avi")

        if not os.path.exists(video_path):
            print(f"Video file not found: {video_path}")
        input_video = ffmpeg.input(video_path)
        print(input_video)
        
        #combine the audio and video
        print('Combining audio and video')
        output_path = os.path.abspath(os.path.join(audiovideo, f"{sessionIndex}_{trialtype}_{trialIndex}_{participant}_{word}_{modality}{correction}_final.avi"))
        ffmpeg.concat(input_video, input_audio, v=1, a=1).output(
            output_path,
            vcodec='libx264',
            acodec='aac',
            video_bitrate='2M',         
            
            ).run(overwrite_output=True)
        
        #save it
        print('Saving the video')


0_1_pr_1_Mic_nominal_srate48000_p0_dansen_geluiden_corrected.csv
0_1_pr_10_Mic_nominal_srate48000_p1_auto_geluiden_corrected.csv
0_1_pr_10_Mic_nominal_srate48000_p1_auto_geluiden_corrected.wav
0_1_pr_18_Mic_nominal_srate48000_p0_snel_combinatie_corrected.csv
0_1_pr_18_Mic_nominal_srate48000_p0_snel_combinatie_corrected.wav
0_1_trial_2_Mic_nominal_srate48000_p0_bitter_geluiden_corrected.csv
0_1_trial_2_Mic_nominal_srate48000_p0_bitter_geluiden_corrected.wav
0_1_trial_8_Mic_nominal_srate48000_p0_walgen_geluiden_corrected.csv
0_1_trial_8_Mic_nominal_srate48000_p0_walgen_geluiden_corrected.wav
0_1_pr_9_Mic_nominal_srate48000_p1_langzaam_geluiden_corrected.csv
0_1_pr_9_Mic_nominal_srate48000_p1_langzaam_geluiden_corrected.wav
0_1_pr_0_Mic_nominal_srate48000_p0_bloem_geluiden.csv
0_1_trial_3_Mic_nominal_srate48000_p0_vechten_geluiden.csv
0_1_trial_3_Mic_nominal_srate48000_p0_vechten_geluiden.wav
Now processing file 0_1_trial_3_Mic_nominal_srate48000_p0_vechten_geluiden.wav
Loading the audio
